In [1]:
EYE_INDICES_TO_LANDMARKS = {

    # Right eye.
    33: [178, 345],
    7: [179, 348],
    163: [178, 352],
    144: [179, 357],
    145: [179, 365],
    153: [179, 371],
    154: [178, 378],
    155: [177, 381],
    133: [177, 383],
    246: [175, 347],
    161: [174, 350],
    160: [172, 355],
    159: [170, 362],
    158: [171, 368],
    157: [172, 375],
    173: [175, 380],
    
    # Left eye.
    263: [176, 467],
    249: [177, 464],
    390: [177, 460],
    373: [178, 455],
    374: [179, 448],
    380: [179, 441],
    381: [178, 435],
    382: [177, 432],
    362: [177, 430],
    466: [175, 465],
    388: [173, 462],
    387: [171, 457],
    386: [170, 450],
    385: [171, 444],
    384: [172, 437],
    398: [175, 432]
}

In [2]:
# EYE 위치정보 저장 
EYE = list(EYE_INDICES_TO_LANDMARKS.keys())
RIGHT_EYE = EYE[:16] 
LEFT_EYE = EYE[16:] 
print(RIGHT_EYE)
print(LEFT_EYE)

[33, 7, 163, 144, 145, 153, 154, 155, 133, 246, 161, 160, 159, 158, 157, 173]
[263, 249, 390, 373, 374, 380, 381, 382, 362, 466, 388, 387, 386, 385, 384, 398]


In [3]:
# 눈의 크기를 재는 함수 
# 유클리디안 거리로 눈꺼풀 사이의 거리 측적 
# 눈을 떴는지 감았는지 판단 

# def cal_area_polygon(coord_list):
    
#     coord_list = np.append(coord_list, np.array(coord_list[0]).reshape(1, 2),axis=0)
    
#     print('check 11111')
#     print(coord_list)
    
#     plus = 0
#     minus = 0
#     for i in range(len(coord_list) - 1):
#         plus += (coord_list[i][0] * coord_list[i+1][1])
#         minus += (coord_list[i][1] * coord_list[i+1][0])
#     area = math.fabs(0.5 * (plus - minus))   
    
#     return area

# def cal_distance(coord_list, direction):
    
#     print('direction :')
#     if direction == 'right':
#         eye_idx = RIGHT_EYE 
#     elif direction == 'left':
#         eye_idx = LEFT_EYE 
    
#     dis_up = coord_list[int(len(eye_idx)/4)][0]**2 + coord_list[int(len(eye_idx)/4 * 3)][0]**2
#     dis_down = coord_list[int(len(eye_idx)/4)][1]**2 + coord_list[int(len(eye_idx)/4 * 3)][1]**2
    
#     print(int(len(eye_idx)/4))
#     print(int(len(eye_idx)/4 *3))
    
#     return abs(dis_up - dis_down)

def euclidean_distance(inst1, inst2):

    # Vector space 

    return np.linalg.norm(inst1 - inst2)

In [ ]:
import cv2
import mediapipe as mp

import numpy as np
import numpy.testing as npt
import math
import time # 시간 모듈 

mp_drawing = mp.solutions.drawing_utils
mp_face_mesh = mp.solutions.face_mesh

global multi_face_landmarks_
global count 

ALERT_SEC = 3 
s_flag = 0 # 눈을 감았을 경우 1 표시 
t_flag = 0 # 눈을 감고 일정 시간이 지난 경우 1 표시 
    
# For static images:
# drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)
# with mp_face_mesh.FaceMesh(
#     static_image_mode=True,
#     max_num_faces=1,
#     min_detection_confidence=0.5) as face_mesh:
#     for idx, file in enumerate(file_list):
#         image = cv2.imread(file)
#         # Convert the BGR image to RGB before processing.
#         results = face_mesh.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

#         # Print and draw face mesh landmarks on the image.
#         if not results.multi_face_landmarks:
#             continue
#         annotated_image = image.copy()
#         for face_landmarks in results.multi_face_landmarks:
#             print('face_landmarks:', face_landmarks)
#             mp_drawing.draw_landmarks(
#                 image=annotated_image,
#                 landmark_list=face_landmarks,
#                 connections=mp_face_mesh.FACE_CONNECTIONS,
#                 landmark_drawing_spec=drawing_spec,
#                 connection_drawing_spec=drawing_spec)
#         cv2.imwrite('/tmp/annotated_image' + str(idx) + '.png', annotated_image)


# For webcam input:
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)
cap = cv2.VideoCapture(0)
with mp_face_mesh.FaceMesh(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as face_mesh:
    
    multi_face_landmarks_ = []
    count = 0 # 초 세기 
    sleep_alert_condition = 0 # 눈을 감고 일정 시간 이상 지속되는지 여부 표시 
    
    while cap.isOpened():
        
        pre_now = time.gmtime(time.time())
        success, image = cap.read()
        if not success:
            print("Ignoring empty camera frame.")
            # If loading a video, use 'break' instead of 'continue'.
            continue

        # Flip the image horizontally for a later selfie-view display, and convert
        # the BGR image to RGB.
        image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
        # To improve performance, optionally mark the image as not writeable to
        # pass by reference.
        image.flags.writeable = False
        results = face_mesh.process(image)

        # Draw the face mesh annotations on the image.
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        multi_face_landmarks_ = []
        right_eye_landmarks = [] # 오른쪽 눈의 x, y 좌표를 담는 리스트 
        left_eye_landmarks = [] # 왼쪽 눈의 x, y 좌표를 담는 리스트 
        
        if results.multi_face_landmarks:
            multi_face_landmarks_ = []
            print('sec : ', count, '\n')
            
            for face_landmarks in results.multi_face_landmarks:    
                mp_drawing.draw_landmarks(
                    image=image,
                    landmark_list=face_landmarks,
                    connections=mp_face_mesh.FACE_CONNECTIONS,
                    landmark_drawing_spec=drawing_spec,
                    connection_drawing_spec=drawing_spec)
                
            for face_landmarks in results.multi_face_landmarks: # 얼굴 전체 좌표를 구함. 
                x = [landmark.x for landmark in face_landmarks.landmark]
                y = [landmark.y for landmark in face_landmarks.landmark]
                face_landmarks = np.transpose(np.stack((y, x))) * image.shape[0:2]
                multi_face_landmarks_.append(face_landmarks)
        
            # 오른쪽 눈 
            for eye_idx in RIGHT_EYE:
#                 print('right eye idx : ', eye_idx)
                eye_landmarks = multi_face_landmarks_[0][eye_idx]
                right_eye_landmarks.append(eye_landmarks)
            right_eye_landmarks = np.array(right_eye_landmarks)

            # 왼쪽 눈 
            for eye_idx in LEFT_EYE: 
#                 print('left eye idx : ', eye_idx)
                eye_landmarks = multi_face_landmarks_[0][eye_idx]
                left_eye_landmarks.append(eye_landmarks)
            left_eye_landmarks = np.array(left_eye_landmarks)
            
            # 유클리디안 거리 
            # 전체 얼굴 크기 구하기 
            face_dis = euclidean_distance(multi_face_landmarks_[0][338], multi_face_landmarks_[0][109])
        
            # 오른쪽 및 왼쪽 눈 크기(눈꺼풀 사이 거리) 구하기
            right_eye_mid_dis = euclidean_distance(right_eye_landmarks[int(len(RIGHT_EYE)/4)], right_eye_landmarks[int(len(RIGHT_EYE)/4) * 3])
            left_eye_mid_dis = euclidean_distance(left_eye_landmarks[int(len(LEFT_EYE)/4)], left_eye_landmarks[int(len(LEFT_EYE)/4) * 3])
            
            # 오른쪽 및 왼쪽 눈을 얼굴 크기로 나눠서 정규화 
            right_eye_mid_dis = int(right_eye_mid_dis / face_dis * 100)
            left_eye_mid_dis = int(left_eye_mid_dis / face_dis * 100)
            
            print('오른쪽 눈 크기 : ', right_eye_mid_dis)
            print('왼쪽 눈 크기 : ', left_eye_mid_dis)
            print()
            
            if (right_eye_mid_dis <= 20) & (left_eye_mid_dis <= 20): # 오른쪽 및 왼쪽 눈 크기가 일정 이하이면 
                print('s_flag !!!!')
                s_flag = 1
            else: 
                s_flag = 0

            # 웹캠에 글자 출력 
            cv2.putText(image, 'face size : ' + str(face_dis), (0, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0))
            cv2.putText(image, 'Right eye size : ' + str(right_eye_mid_dis), (0, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0))
            cv2.putText(image, 'Left eye size : ' + str(left_eye_mid_dis), (0, 200), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0))
            
            cv2.putText(image, 'sec : ' + str(count), (0, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0))

            if sleep_alert_condition == 1: # 눈을 감고 일정 시간이상 지속되는 경우 경고 표시 
                cv2.putText(image, 'WARNING!!! Please Look ahead!!! ' + str(t_flag), (0, 300), cv2.FONT_HERSHEY_DUPLEX, 1, (0, 0, 255))
            
            cv2.imshow('MediaPipe FaceMesh', image)
        
        if s_flag == 1: # 눈 감은 상태 
            t_flag += 1
        else: 
            s_flag = 0
            t_flag = 0
            sleep_alert_condition = 0
        
        if t_flag >= ALERT_SEC: # 눈을 감고 ALERT_SEC 초 이상 유지
            sleep_alert_condition = 1
        else: 
            sleep_alert_condition = 0 
        
        print('correct condition ? ') 
        print('s_flag ', s_flag, 't_flag ', t_flag, 'sleep_alert_condition ', sleep_alert_condition)
        print()
        
        time.sleep(1) # 1초 delay, while문 한번 도는데 1초 걸림.  
        count += 1
        
        if cv2.waitKey(5) & 0xFF == 27:
            break
        if cv2.waitKey(20) & 0xFF == 0x1B: # 무한 반복중, 키보드 ESC 아스키코드가 들어오면 빠져나감. break
            break
    cv2.destroyAllWindows() # 창을 종료

cap.release()

correct condition ? 
s_flag  0 t_flag  0 sleep_alert_condition  0

sec :  1 

오른쪽 눈 크기 :  23
왼쪽 눈 크기 :  22

correct condition ? 
s_flag  0 t_flag  0 sleep_alert_condition  0

sec :  2 

오른쪽 눈 크기 :  26
왼쪽 눈 크기 :  26

correct condition ? 
s_flag  0 t_flag  0 sleep_alert_condition  0

sec :  3 

오른쪽 눈 크기 :  30
왼쪽 눈 크기 :  29

correct condition ? 
s_flag  0 t_flag  0 sleep_alert_condition  0

sec :  4 

오른쪽 눈 크기 :  26
왼쪽 눈 크기 :  25

correct condition ? 
s_flag  0 t_flag  0 sleep_alert_condition  0

sec :  5 

오른쪽 눈 크기 :  26
왼쪽 눈 크기 :  26

correct condition ? 
s_flag  0 t_flag  0 sleep_alert_condition  0

sec :  6 

오른쪽 눈 크기 :  26
왼쪽 눈 크기 :  25

correct condition ? 
s_flag  0 t_flag  0 sleep_alert_condition  0

sec :  7 

오른쪽 눈 크기 :  22
왼쪽 눈 크기 :  23

correct condition ? 
s_flag  0 t_flag  0 sleep_alert_condition  0

sec :  8 

오른쪽 눈 크기 :  25
왼쪽 눈 크기 :  24

correct condition ? 
s_flag  0 t_flag  0 sleep_alert_condition  0

sec :  9 

오른쪽 눈 크기 :  23
왼쪽 눈 크기 :  23

correct condition ? 
s_flag 